In [1]:
%pwd

'/home/by783/Self_Jupyter/DL_Final_Project'

# data mean and std
Ref: https://discuss.pytorch.org/t/computing-the-mean-and-std-of-dataset/34949



In [59]:

import torch
import torch.nn as nn
import torchvision

import argparse
import os
import numpy as np
import torchvision
import torchvision.transforms as transforms

https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457/11

In [60]:
from __future__ import print_function
from __future__ import division

import numpy as np
import argparse
import random
import shutil
import time
import warnings


import torch
import torch.nn as nn

import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim as optim

import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed

import torchvision
from torchvision import datasets, models, transforms
import os
import sys
import copy

In [14]:
batch_size=128

In [15]:
loader_image_path='/scratch/by783/DL_Final/ssl_data_96'

transform = transforms.Compose([transforms.ToTensor()])

sup_train_data = datasets.ImageFolder('{}/{}/train'.format(loader_image_path, 'supervised'), transform=transform)
sup_val_data = datasets.ImageFolder('{}/{}/val'.format(loader_image_path, 'supervised'), transform=transform)
unsup_data = datasets.ImageFolder('{}/{}/'.format(loader_image_path, 'unsupervised'), transform=transform)

data_loader_sup_train = torch.utils.data.DataLoader(
        sup_train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
data_loader_sup_val = torch.utils.data.DataLoader(
        sup_val_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
data_loader_unsup = torch.utils.data.DataLoader(
        unsup_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )

In [21]:
len(data_loader_sup_train.dataset)

64000

In [23]:
loader = data_loader_sup_train

mean = 0.
std = 0.
i=0
sample_num=0

for images, _ in loader:
    #if i%10==0: print(10)
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    
    sample_num+=batch_samples
    print('mean: ',mean/sample_num)
    print('std: ',std/sample_num)

mean /= len(loader.dataset)
std /= len(loader.dataset)

mean:  tensor([0.4980, 0.4755, 0.4279])
std:  tensor([0.2300, 0.2242, 0.2246])
mean:  tensor([0.5002, 0.4791, 0.4320])
std:  tensor([0.2297, 0.2236, 0.2270])
mean:  tensor([0.4999, 0.4775, 0.4313])
std:  tensor([0.2280, 0.2230, 0.2264])
mean:  tensor([0.5034, 0.4780, 0.4287])
std:  tensor([0.2282, 0.2226, 0.2247])
mean:  tensor([0.5003, 0.4741, 0.4256])
std:  tensor([0.2260, 0.2209, 0.2226])
mean:  tensor([0.5014, 0.4752, 0.4286])
std:  tensor([0.2262, 0.2225, 0.2242])
mean:  tensor([0.5014, 0.4772, 0.4305])
std:  tensor([0.2255, 0.2226, 0.2252])
mean:  tensor([0.5019, 0.4773, 0.4305])
std:  tensor([0.2258, 0.2222, 0.2253])
mean:  tensor([0.5014, 0.4758, 0.4289])
std:  tensor([0.2270, 0.2233, 0.2261])
mean:  tensor([0.5037, 0.4784, 0.4309])
std:  tensor([0.2261, 0.2227, 0.2258])
mean:  tensor([0.5021, 0.4778, 0.4295])
std:  tensor([0.2260, 0.2227, 0.2257])
mean:  tensor([0.5004, 0.4751, 0.4259])
std:  tensor([0.2261, 0.2228, 0.2258])
mean:  tensor([0.5011, 0.4746, 0.4254])
std:  tensor

KeyboardInterrupt: 

In [24]:
loader = data_loader_sup_val

mean = 0.
std = 0.
i=0
sample_num=0

for images, _ in loader:
    #if i%10==0: print(10)
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    
    sample_num+=batch_samples
    print('mean: ',mean/sample_num)
    print('std: ',std/sample_num)

mean /= len(loader.dataset)
std /= len(loader.dataset)

mean:  tensor([0.5197, 0.4802, 0.4259])
std:  tensor([0.2335, 0.2296, 0.2397])
mean:  tensor([0.5198, 0.4824, 0.4336])
std:  tensor([0.2308, 0.2277, 0.2368])
mean:  tensor([0.5122, 0.4791, 0.4254])
std:  tensor([0.2262, 0.2243, 0.2307])
mean:  tensor([0.5133, 0.4819, 0.4311])
std:  tensor([0.2265, 0.2241, 0.2296])
mean:  tensor([0.5090, 0.4750, 0.4238])
std:  tensor([0.2243, 0.2210, 0.2261])
mean:  tensor([0.5073, 0.4742, 0.4228])
std:  tensor([0.2257, 0.2218, 0.2267])
mean:  tensor([0.5041, 0.4725, 0.4224])
std:  tensor([0.2258, 0.2219, 0.2276])
mean:  tensor([0.5047, 0.4739, 0.4239])
std:  tensor([0.2260, 0.2224, 0.2279])
mean:  tensor([0.5044, 0.4740, 0.4245])
std:  tensor([0.2257, 0.2223, 0.2276])
mean:  tensor([0.5041, 0.4739, 0.4243])
std:  tensor([0.2260, 0.2226, 0.2278])
mean:  tensor([0.5031, 0.4722, 0.4229])
std:  tensor([0.2266, 0.2233, 0.2283])
mean:  tensor([0.5029, 0.4715, 0.4215])
std:  tensor([0.2265, 0.2230, 0.2278])
mean:  tensor([0.5035, 0.4723, 0.4228])
std:  tensor

limit_output extension: Maximum message size of 4000 exceeded with 4029 characters

KeyboardInterrupt: 

In [25]:
loader = data_loader_unsup

mean = 0.
std = 0.
i=0
sample_num=0

for images, _ in loader:
    #if i%10==0: print(10)
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    
    sample_num+=batch_samples
    print('mean: ',mean/sample_num)
    print('std: ',std/sample_num)

mean /= len(loader.dataset)
std /= len(loader.dataset)

mean:  tensor([0.5099, 0.4805, 0.4300])
std:  tensor([0.2306, 0.2213, 0.2273])
mean:  tensor([0.4942, 0.4704, 0.4167])
std:  tensor([0.2238, 0.2170, 0.2218])
mean:  tensor([0.4969, 0.4716, 0.4221])
std:  tensor([0.2256, 0.2199, 0.2235])
mean:  tensor([0.5024, 0.4748, 0.4235])
std:  tensor([0.2249, 0.2204, 0.2240])
mean:  tensor([0.5040, 0.4758, 0.4246])
std:  tensor([0.2240, 0.2194, 0.2225])
mean:  tensor([0.5011, 0.4730, 0.4215])
std:  tensor([0.2237, 0.2193, 0.2221])
mean:  tensor([0.5031, 0.4753, 0.4225])
std:  tensor([0.2226, 0.2184, 0.2211])
mean:  tensor([0.5005, 0.4732, 0.4214])
std:  tensor([0.2237, 0.2195, 0.2224])
mean:  tensor([0.5022, 0.4748, 0.4225])
std:  tensor([0.2223, 0.2191, 0.2222])
mean:  tensor([0.5027, 0.4753, 0.4232])
std:  tensor([0.2218, 0.2187, 0.2223])
mean:  tensor([0.5037, 0.4752, 0.4231])
std:  tensor([0.2224, 0.2193, 0.2230])
mean:  tensor([0.5033, 0.4739, 0.4217])
std:  tensor([0.2227, 0.2197, 0.2239])
mean:  tensor([0.5026, 0.4733, 0.4216])
std:  tensor

limit_output extension: Maximum message size of 4000 exceeded with 4029 characters

KeyboardInterrupt: 

In [26]:
# test normalization
loader_image_path='/scratch/by783/DL_Final/ssl_data_96'

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.502, 0.474, 0.426], [0.227, 0.222, 0.226])])

sup_train_data = datasets.ImageFolder('{}/{}/train'.format(loader_image_path, 'supervised'), transform=transform)
sup_val_data = datasets.ImageFolder('{}/{}/val'.format(loader_image_path, 'supervised'), transform=transform)
unsup_data = datasets.ImageFolder('{}/{}/'.format(loader_image_path, 'unsupervised'), transform=transform)

data_loader_sup_train = torch.utils.data.DataLoader(
        sup_train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
data_loader_sup_val = torch.utils.data.DataLoader(
        sup_val_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
data_loader_unsup = torch.utils.data.DataLoader(
        unsup_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )


loader = data_loader_sup_train

mean = 0.
std = 0.
i=0
sample_num=0

for images, _ in loader:
    #if i%10==0: print(10)
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    
    sample_num+=batch_samples
    print('mean: ',mean/sample_num)
    print('std: ',std/sample_num)

mean /= len(loader.dataset)
std /= len(loader.dataset)

mean:  tensor([0.0257, 0.0389, 0.0037])
std:  tensor([0.9625, 0.9704, 0.9698])
mean:  tensor([-0.0127, -0.0253, -0.0596])
std:  tensor([1.0078, 1.0028, 0.9835])
mean:  tensor([-0.0210, -0.0042, -0.0258])
std:  tensor([1.0049, 1.0044, 0.9886])
mean:  tensor([-0.0077, -0.0018, -0.0057])
std:  tensor([1.0046, 1.0038, 0.9944])
mean:  tensor([-0.0103, -0.0061, -0.0140])
std:  tensor([1.0029, 1.0052, 0.9978])
mean:  tensor([-0.0208, -0.0161, -0.0172])
std:  tensor([0.9994, 1.0022, 1.0003])
mean:  tensor([-0.0079, -0.0078, -0.0151])
std:  tensor([1.0054, 1.0048, 1.0045])
mean:  tensor([-0.0255, -0.0241, -0.0276])
std:  tensor([1.0071, 1.0058, 1.0025])
mean:  tensor([-0.0085, -0.0162, -0.0157])
std:  tensor([1.0105, 1.0101, 1.0085])
mean:  tensor([-0.0165, -0.0210, -0.0169])
std:  tensor([1.0082, 1.0077, 1.0060])
mean:  tensor([-0.0172, -0.0224, -0.0163])
std:  tensor([1.0058, 1.0074, 1.0065])
mean:  tensor([-0.0231, -0.0257, -0.0194])
std:  tensor([1.0042, 1.0080, 1.0061])
mean:  tensor([-0.0

KeyboardInterrupt: 

Hooks

In [64]:
class TEST(torch.nn.Module):
    def __init__(self):
        super(TEST, self).__init__()
        self.encoder=nn.Sequential(nn.Linear(1,1))
        self.decoder=nn.Sequential(nn.Linear(1,1))
        
    def forward(self,x):
        x=self.encoder(x)
        x=self.decoder(x)
        return x

def Get_features(self, _input, _output):
    print('print: input:',_input)
    print('print: output:',_output)
    #return output

def Get_encoder_features(self, input_, output_):
    encoder_output.append(output_)
    
def Get_decoder_features(self, input_, output_):
    encoder_output.append(input_)

In [65]:
x=torch.tensor([[1.0]])

In [66]:
model=TEST()

# 如何重设 layer
# https://discuss.pytorch.org/t/load-initial-weights-to-one-layer-of-the-convnet-manually/13336
model.encoder[0].weight.data = torch.tensor([[2.0]])
model.encoder[0].bias.data=torch.tensor([[0.0]])


model.decoder[0].weight.data = torch.tensor([[3.0]])
model.decoder[0].bias.data=torch.tensor([[0.0]])

In [67]:
encoder_output=[]
model.decoder[0].register_forward_hook(Get_decoder_features)

In [68]:
l=model(x)

In [69]:
encoder_output

[(tensor([[2.]], grad_fn=<ThAddmmBackward>),)]

In [70]:
l.backward()

In [71]:
model.encoder[0].weight.grad

tensor([[3.]])

In [72]:
model.decoder[0].weight.grad

tensor([[2.]])

In [103]:
model2=TEST()

# 如何重设 layer
# https://discuss.pytorch.org/t/load-initial-weights-to-one-layer-of-the-convnet-manually/13336
model2.encoder[0].weight.data = torch.tensor([[2.0]])
model2.encoder[0].bias.data=torch.tensor([[0.0]])


model2.decoder[0].weight.data = torch.tensor([[3.0]])
model2.decoder[0].bias.data=torch.tensor([[0.0]])

In [104]:
encoder_output=[]
model2.decoder[0].register_forward_hook(Get_decoder_features)

In [105]:
l=model2(x)

In [109]:
encoder_output[0]

(tensor([[2.]], grad_fn=<ThAddmmBackward>),)

In [108]:
l

tensor([[6.]], grad_fn=<ThAddmmBackward>)

In [107]:
l=l+encoder_output[0]

TypeError: add() received an invalid combination of arguments - got (tuple), but expected one of:
 * (Tensor other, Number alpha)
 * (Number other, Number alpha)


In [102]:
l.backward()

In [44]:
model2.encoder[0].weight.grad

tensor([[4.]])

In [45]:
model2.decoder[0].weight.grad

tensor([[2.]])

# the hook works, can extract the layer vaild layer output ('True' layer output, which remember its position in the computational graph and correctly calculate the grad) 

In [19]:
import sys
sys.exit('stop here')

SystemExit: stop here

/Users/Bixing/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
en_output

In [ ]:
model.encoder[0].weight.data

In [ ]:
l=model(x)

In [ ]:
l.backward()

In [ ]:
model.decoder[0].weight.grad

In [ ]:
de_layer=model.decoder[0]

In [ ]:
en_layer=model.encoder[0]

In [ ]:

de_layer.register_forward_hook(Get_features)

In [ ]:
l=model(x)

In [27]:
a=[1,2,3,4,5]

In [29]:
a[::-1]

[5, 4, 3, 2, 1]